In [ ]:
PAGE REPLACEMENT POLICIES


In [ ]:
#FIFO
def fifo(pages , capacity) :
    cache = []
    faults = 0
    for page in pages:
        if page not in cache:
            if len(cache) >= capacity :
                cache.pop(0)
            cache.append(page)
            faults += 1
    return faults

In [ ]:
#LRU
def lru(pages , capacity) :
    cache = []
    faults = 0
    for page in pages :
        if page  in cache :
            cache.remove(page)
        else:
            if len(cache) >= capacity :
                cache.pop(0)
            faults +=1
            cache.append(page)
    return faults

In [ ]:
#optimal
def optimal(pages , capacity) :
    cache = []
    faults = 0
    for i,page in enumerate(pages) :
        if page not in cache :
            if len(cache) >= capacity :
                furthest = max(cache , key = lambda x : pages[i + 1:].index(x) if x in pages[i + 1:] else float('inf'))
                cache.remove(furthest)
            cache.append(page)
            faults += 1
    return faults
    

In [ ]:
pages = [7, 0, 1, 2, 0, 3, 0, 4, 2, 3, 0, 3, 2]
capacity = 4

print("FIFO Faults:", fifo(pages, capacity))
print("LRU Faults:", lru(pages, capacity))
print("Optimal Faults:", optimal(pages, capacity))

BANKER'S ALGORITHM

In [20]:
def bankers_algorithm( alloc , max_need , avail) :
    n = len(alloc)
    m = len(avail)
    need = [[max_need[i][j] - alloc[i][j] for j in range(m)] for i in range (n)]
    finish = [False] * n
    safe_seq = []
    def can_allocate(process) :
        return all(need[process][j] <= avail[j] for j in range(m))
    while len(safe_seq) < n :
        allocated = False
        for i in range(n) :
            if not finish[i] and can_allocate(i) :
                avail = [avail[j] + alloc[i][j] for j in range(m)]
                finish[i] = True
                safe_seq = safe_seq + [i]
                allocated = True
                break
        if not allocated :
            return "Unsafe state"
    return safe_seq

In [21]:
alloc = [[0, 1, 0], [2, 0, 0], [3, 0, 2], [2, 1, 1], [0, 0, 2]]
max_need = [[7, 5, 3], [3, 2, 2], [9, 0, 2], [2, 2, 2], [4, 3, 3]]
avail = [3, 3, 2]

print(bankers_algorithm(alloc, max_need, avail))


[1, 3, 0, 2, 4]


CACHE MEMORY MAPPING

In [22]:
def fully_associative(addresses , cache_size) :
    cache , hits , accesses = [] , 0 , len(addresses)
    for addr in addresses :
        if addr in cache :
            hit += 1
        elif len(cache) < cache_size :
            cache.append(addr);
        else :
            cache.pop(0)
            cache.append(addr)
    return hits , accesses

In [23]:
addresses = [0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7]
cache_size = 4

In [26]:
hits , accesses = fully_associative(addresses , cache_size)
hit_rate = hits/accesses
print(hits , accesses , hit_rate)

0 16 0.0


MEMORY ALLOCATION POLICY

In [31]:
def allocate(blockSize , process_size , stratergy) :
    allocation = [-1]* len(processSize)
    for i , process in enumerate(processSize) :
        if stratergy == "first" :
            for j , block in enumerate(blockSize) :
                if block >= process :
                    allocation[i] = j + 1
                    blockSize[j] = process
                    break
        elif stratergy == "best" :
            best = -1
            for j , block in enumerate(blockSize) :
                if block >= process and (best == -1 or block < blockSize[best]) :
                    best = j
            if best != -1 :
                allocation[i] = best + 1
                blockSize[best] = process
        elif stratergy == "worst" :
            worst = -1
            for j , block in enumerate(blockSize) :
                if block >= process and (worst == -1 or block > blockSize[worst]):
                    worst = j
            if worst != -1 :
                allocation[i] = worst + 1
                blockSize [worst] -= process
    return allocation

blockSize = [100, 500, 200, 300, 600]
processSize = [212, 417, 112, 426]

print("First Fit:", allocate(blockSize[:], processSize, "first"))
print("Best Fit:", allocate(blockSize[:], processSize, "best"))
print("Worst Fit:", allocate(blockSize[:], processSize, "worst"))


First Fit: [2, 5, 2, -1]
Best Fit: [4, 2, 3, 5]
Worst Fit: [5, 2, 5, -1]


NON-PREEMPTIVE SCHEDULING

In [18]:
#FCFS
def fcfs(processes , arrival, burst) :
    n = len(processes)
    start , finish , wait , tat = [0] * n , [0] * n , [0] * n , [0] * n
    for i in range(n) :
        start[i] = max(finish[i-1] if i>0 else 0 , arrival[i])
        finish[i] = start[i] + burst[i]
        wait[i] = start[i] - arrival[i]
        tat[i] = finish[i] -arrival[i]
    avg_wait = sum(wait)/n
    avg_tat = sum(tat)/n

    print("\nGANTT CHART :")
    print("|".join(f"P{processes[i]}" for i in range(n)))
    print("0"," ".join(str(finish[i]) for i in range(n)))
    return avg_wait , avg_tat
processes = [1, 2, 3]
arrival = [0, 1, 2]
burst = [5, 3, 8]
avg_wait, avg_tat = fcfs(processes, arrival, burst)
print(f"Average Waiting Time: {avg_wait}")
print(f"Average Turnaround Time: {avg_tat}")


GANTT CHART :
P1|P2|P3
0 5 8 16
Average Waiting Time: 3.3333333333333335
Average Turnaround Time: 8.666666666666666


In [26]:
def sjn(processes, arrival, burst):
    n=len(processes)
    time, finish, wait, tat = 0, [0] * n, [0] * n, [0] * n
    remaining = list(range(n))
    gantt = []
    while remaining:
        available = [i for i in remaining if arrival[i] <= time]
        if not available:
            time += 1
            continue
        shortest = min(available, key=lambda i: burst[i])
        gantt.append(shortest)
        time = max(time, arrival[shortest]) + burst[shortest]
        finish[shortest] = time
        wait[shortest] = time - arrival[shortest] - burst[shortest]
        tat[shortest] = finish[shortest] - arrival[shortest]
        remaining.remove(shortest)

    avg_wait = sum(wait) / n
    avg_tat = sum(tat) / n

    print("\nGantt Chart:")
    print(" | ".join(f"P{processes[i]}" for i in gantt))
    print("0", " ".join(str(finish[i]) for i in gantt))

    return avg_wait, avg_tat

# Example
processes = [1, 2, 3]
arrival = [0, 1, 2]
burst = [5, 3, 8]
avg_wait, avg_tat = sjn(processes, arrival, burst)
print(f"Average Waiting Time: {avg_wait}")
print(f"Average Turnaround Time: {avg_tat}")



Gantt Chart:
P1 | P2 | P3
0 5 8 16
Average Waiting Time: 3.3333333333333335
Average Turnaround Time: 8.666666666666666


In [34]:
#priority scheduling
def priority_scheduling( processes , arrival , burst , priority) :
    n = len(processes)
    time = 0
    finish , wait , tat = [0] * n , [0] * n , [0] * n
    remaining = list(range(n))
    gantt = []
    while remaining :
        available = [i for i in remaining if arrival[i] <= time]
        if not available :
            time += 1
            continue
        highest_priority = min(available , key=lambda i : priority[i])
        gantt.append(highest_priority)
        time = max(time , arrival[highest_priority]) + burst[highest_priority]
        finish[highest_priority] = time
        wait[highest_priority] = time - arrival[highest_priority] - burst[highest_priority]
        tat[highest_priority] = finish[highest_priority] - arrival[highest_priority]
        remaining.remove(highest_priority)
    avg_wait = sum(wait) / n
    avg_tat = sum(tat) / n

    print("\nGantt Chart:")
    print(" | ".join(f"P{processes[i]}" for i in gantt))
    print("0", " ".join(str(finish[i]) for i in gantt))

    return avg_wait, avg_tat

# Example
processes = [1, 2, 3]
arrival = [0, 1, 2]
burst = [5, 3, 8]
priority = [2, 1, 3]
avg_wait, avg_tat = priority_scheduling(processes, arrival, burst, priority)
print(f"Average Waiting Time: {avg_wait}")
print(f"Average Turnaround Time: {avg_tat}")
    


Gantt Chart:
P1 | P2 | P3
0 5 8 16
Average Waiting Time: 3.3333333333333335
Average Turnaround Time: 8.666666666666666


PREEMPTIVE SCHEDULING

In [6]:
#SRTF
def srtf( processes , arrival , burst) :
    n = len(processes)
    remaining = burst[:]
    time , completed = 0 , 0
    wait , tat = [0] * n , [0] * n
    gantt = []
    while completed < n :
        available = [i for i in range(n) if arrival[i] <= time and remaining[i] > 0]
        if available :
            current = min(available , key=lambda i : remaining [i])
            gantt.append(processes[current])
            remaining[current] -= 1
            if remaining[current] == 0 :
                completed += 1
                finish_time = time + 1
                tat[current] = finish_time - arrival[current]
                wait[current] = tat[current] - burst[current]
        time += 1
    avg_wait = sum(wait)/n
    avg_tat = sum(tat)/n
    print("\nGANTT CHART :")
    print("|".join(map(str,gantt)))
    print(f"Average Waiting Time: {avg_wait}")
    print(f"Average Turnaround Time: {avg_tat}")

# Example
processes = [1, 2, 3]
arrival = [0, 2, 4]
burst = [7, 4, 1]
srtf(processes, arrival, burst)


GANTT CHART :
1|1|2|2|3|2|2|1|1|1|1|1
Average Waiting Time: 2.0
Average Turnaround Time: 6.0


In [7]:
#priority
def preemptive_priority(processes , arrival , burst , priority) :
    n = len(processes)
    remaining = burst[:]
    time , completed = 0 , 0
    wait , tat = [0] * n , [0] * n
    gantt =[]
    while completed < n :
        available = [i for i in range(n) if arrival[i] <= time and remaining[i] > 0]
        if available :
            current = min(available , key=lambda i : priority[i])
            gantt.append(processes[current])
            remaining[current] -= 1
            if remaining[current] == 0 :
                completed += 1
                finish_time = time + 1
                tat[current] = finish_time - arrival[current]
                wait[current] = tat[current] - burst[current]
        time += 1
    avg_wait = sum(wait) / n
    avg_tat = sum(tat) / n
    print("\nGantt Chart:")
    print(" | ".join(map(str, gantt)))
    print(f"Average Waiting Time: {avg_wait}")
    print(f"Average Turnaround Time: {avg_tat}")

# Example
processes = [1, 2, 3]
arrival = [0, 1, 2]
burst = [8, 4, 2]
priority = [2, 1, 3]
preemptive_priority(processes, arrival, burst, priority)    


Gantt Chart:
1 | 2 | 2 | 2 | 2 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 3 | 3
Average Waiting Time: 4.666666666666667
Average Turnaround Time: 9.333333333333334


In [ ]:
#Round Robin
def round_robin(processes , arrival , burst , quantum) :
    n = len(processes)
    remaining = burst[:]
    time , completed = 0 , 0
    wait , tat = [0] * n , [0] * n
    queue = []
    gantt = []
    while completed < n or queue :
        for i in range(n) :
            if arrival[i] <= time and i not in queue and remaining[i] > 0 :
                queue.append(i)
        if queue :
            current = queue.pop(0)
            gantt.append(processes[current])
            executed_time = min(remaining[current] , quantum)
            remaining[current] -= executed_time
            time += executed_time
            if remaining[current] > 0 :
                queue.append(current)
            else :
                completed += 1
                tat[current] = time - arrival[current]
                wait[current] = tat[current] - burst[current]
        else :
            time += 1
    avg_wait = sum(wait) / n
    avg_tat = sum(tat) / n

    print("\nGantt Chart:")
    print(" | ".join(map(str, gantt)))
    print(f"Average Waiting Time: {avg_wait}")
    print(f"Average Turnaround Time: {avg_tat}")

# Example
processes = [1, 2, 3]
arrival = [0, 1, 2]
burst = [5, 4, 2]
quantum = 2
round_robin(processes, arrival, burst, quantum)

PRODUCER-CONSUMER PROBLEM

In [1]:
import threading
import time
import random

buffer_size = 10
num_items = 20

empty = threading.Semaphore(buffer_size)
full = threading.Semaphore(0)
mutex = threading.Semaphore(1)

buffer = []
produced_count = 0
consumed_count = 0

def producer() :
    global produced_count
    while produced_count < num_items :
        item = random.randint(1 , 100)
        empty.acquire()
        mutex.acquire()
        if produced_count < num_items :
            buffer.append(item)
            produced_count += 1
            print(f"Produced : {item} , Buffer: {buffer}")
        mutex.release()
        full.release()
        time.sleep(random.uniform(0.5 , 2))

def consumer() :
    global consumed_count
    while consumed_count < num_items :
        full.acquire()
        mutex.acquire()
        if consumed_count < num_items :
            item = buffer.pop(0)
            consumed_count += 1
            print(f"Consumed : {item} , Buffer : {buffer}")
        mutex.release()
        empty.release()
        time.sleep(random.uniform(0.5 , 2))

producer_thread = threading.Thread(target = producer)
consumer_thread = threading.Thread(target = consumer)

producer_thread.start()
consumer_thread.start()

producer_thread.join()
consumer_thread.join()

print("\n All items have been consumed")

Produced : 82 , Buffer: [82]
Consumed : 82 , Buffer : []
Produced : 71 , Buffer: [71]
Consumed : 71 , Buffer : []
Produced : 84 , Buffer: [84]
Consumed : 84 , Buffer : []
Produced : 90 , Buffer: [90]
Consumed : 90 , Buffer : []
Produced : 26 , Buffer: [26]
Consumed : 26 , Buffer : []
Produced : 26 , Buffer: [26]
Consumed : 26 , Buffer : []
Produced : 32 , Buffer: [32]
Consumed : 32 , Buffer : []
Produced : 62 , Buffer: [62]
Consumed : 62 , Buffer : []
Produced : 38 , Buffer: [38]
Consumed : 38 , Buffer : []
Produced : 57 , Buffer: [57]
Consumed : 57 , Buffer : []
Produced : 34 , Buffer: [34]
Consumed : 34 , Buffer : []
Produced : 58 , Buffer: [58]
Produced : 96 , Buffer: [58, 96]
Consumed : 58 , Buffer : [96]
Consumed : 96 , Buffer : []
Produced : 98 , Buffer: [98]
Consumed : 98 , Buffer : []
Produced : 21 , Buffer: [21]
Consumed : 21 , Buffer : []
Produced : 18 , Buffer: [18]
Produced : 9 , Buffer: [18, 9]
Consumed : 18 , Buffer : [9]
Produced : 89 , Buffer: [9, 89]
Consumed : 9 , Buf

DISK SCHEDULING

In [2]:
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
#FCFS
def fcf(requests , head) :
  seq = [head] + requests
  return seq

requests = [98, 183, 37, 122, 14, 124, 65, 67]
head = 53
seq = fcf(requests , head)

plt.figure(figsize = ( 8 , 5 ))
plt.plot(seq , range(len(seq)) , marker = 'o' , linestyle ='-' , color = 'b')
plt.title("FCFS")
plt.xlabel("Disk cylinder")
plt.ylabel("Request Order")
plt.gca().invert_yaxis()
plt.grid(True)
plt.show()

In [ ]:
#SSTF
def sstf(req , head) :
  seq = [head]
  pending = req[:]
  while pending :
    nearest = min(pending, key=lambda x : abs(x-head))
    seq.append(nearest)
    pending.remove(nearest)
  return seq
req = [98, 183, 37, 122, 14, 124, 65, 67]
head = 53
seq = sstf(req , head)

plt.figure(figsize=(8, 5))
plt.plot(seq, range(len(seq)), marker='o', linestyle='-', color='b')
plt.title("SSTF Disk Scheduling")
plt.xlabel("Disk Cylinder")
plt.ylabel("Request Order")
plt.gca().invert_yaxis()
plt.grid(True)
plt.show()

In [ ]:
#SCAN
def scan(req , head , dirctn , size) :
  seq = [head]
  pending = sorted(req)
  if dirctn == "left" :
    for reqs in reversed(pending) :
      if reqs <= head :
        seq.append(reqs)
        head = 0
        for reqs in pending :
          if reqs > head :
            seq.append(req)
  else :
    for reqs in pending :
      if reqs >= head :
        seq.append(reqs)
    seq.append(size - 1)
    head = size - 1
    for reqs in reversed(pending) :
      if reqs < head :
        seq.append(reqs)
  return seq

requests = [98, 183, 37, 122, 14, 124, 65, 67]
head = 53
direction = "left"
disk_size = 200
sequence = scan(requests, head, direction, disk_size)

plt.figure(figsize=(8,5))
plt.plot(seq , range(len(seq)) , marker = 'o' , linestyle = '-' , color = 'b')
plt.title("SCAN")
plt.xlabel("Disk cylinder")
plt.ylabel("Request order")
plt.gca().invert_yaxis()
plt.grid(True)
plt.show()

In [ ]:
#C-SCAN
def cscan(req , head , size) :
  seq = [head]
  pending = sorted(req)
  for reqs in pending :
    if reqs >= head :
      seq.append(reqs)
  seq.append(size - 1)
  seq.append(0)
  for reqs in pending :
    if reqs < head :
      seq.append(reqs)
    return seq

req = [98, 183, 37, 122, 14, 124, 65, 67]
head = 53
size = 200
seq = cscan(req, head, size)

plt.figure(figsize=(8, 5))
plt.plot(seq, range(len(seq)), marker='o', linestyle='-', color='b')
plt.title("C-SCAN Disk Scheduling")
plt.xlabel("Disk Cylinder")
plt.ylabel("Request Order")
plt.gca().invert_yaxis()
plt.grid(True)
plt.show()

In [ ]:
#LOOK
def look(req , head , dirctn) :
  seq = [head]
  pending = sorted(req)
  if dirctn == "left" :
    for reqs in reversed(pending) :
      if reqs <= head :
        seq.append(reqs)
    for reqs in pending :
      if reqs > head :
        seq.append(reqs)
  else :
    for reqs in pending :
      if reqs >= head :
        seq.append(reqs)
      for reqs in reversed(pending) :
        if reqs < head :
          seq.append(reqs)
  return seq

requests = [98, 183, 37, 122, 14, 124, 65, 67]
head = 53
dirctn = "left"
seq = look(req, head, dirctn)

plt.figure(figsize=(8, 5))
plt.plot(seq, range(len(seq)), marker='o', linestyle='-', color='b')
plt.title("LOOK Disk Scheduling")
plt.xlabel("Disk Cylinder")
plt.ylabel("Request Order")
plt.gca().invert_yaxis()
plt.grid(True)
plt.show()

In [ ]:
#C-LOOK
def clook(req , head) :
  seq = [head]
  pending = sorted(req)
  for reqs in pending :
    if reqs >= head :
      seq.append(reqs)
  seq.append(pending[0])
  for reqs in pending :
    if reqs < head :
      seq.append(reqs)
  return seq

req = [98, 183, 37, 122, 14, 124, 65, 67]
head = 53
seq = clook(req, head)

plt.figure(figsize=(8, 5))
plt.plot(seq, range(len(seq)), marker='o', linestyle='-', color='b')
plt.title("C-LOOK Disk Scheduling")
plt.xlabel("Disk Cylinder")
plt.ylabel("Request Order")
plt.gca().invert_yaxis()
plt.grid(True)
plt.show()